# Chapter 5. Sinuoidal fluctuation of river stage

IHE, Delft, 2017-12-20, 28-12-2020

@T.N.Olsthoorn


## A river whose level fluctuates according to a sine function

The analytical solution is given by (see syllabus)

$$s = A e^{-a x} \sin(\omega t - a x + \theta)$$

Give the envelopes between which the head will fluctuate due to only this wavy head fluctuations at $x=0$

Show the wave in the aquifer at a number of times using the following variable values:

$kD = 500\,  m2/d$, the aquifer's transmissivity

$S = 0.001$, the aquifer's storage coefficient

$A  = 2 \, m$, the wave's amplitude

$\omega = 1/\pi \, d^{-1}$, the wave's frequency

$\theta = 0.2/\pi$, the wave's initial time delay

## importing modules

In [118]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import PathPatch, Path
from IPython.display import HTML
import pdb

## Convenience function to set up a graphic

In [11]:
def newfig(title='?', xlabel='?', ylabel='?', xlim=None, ylim=None,
                   xscale='linear', yscale='linear', size_inches=(14, 8), fontsize=15):
    '''Setup a new axis for plotting'''
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if xscale: ax.set_xscale(xscale)
    if yscale: ax.set_yscale(yscale)
    if xlim: ax.set_xlim(xlim)
    if ylim: ax.set_ylim(ylim)
    ax.grid(True, which='both')

    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                     ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(fontsize)
    return fig, ax

## Implementation

In [228]:
# define the variables and their values
Tc    = 0.5 # [d] cycle time
omega = 2 * np.pi / Tc  # [1/d] angle velocity randians/day
theta = 0.2 * np.pi    # [-]  initial time delay in radians
S     = 0.001           # [-]   storage coefficient of aquifer
kD    = 1000           # m2/d, transmissivity
A     = 2              # m, sine ampletude
a     = np.sqrt(omega / 2 * S / kD) # 1/m, damping and delay factor
x     = np.linspace(0, 2000, 501)  # m, choose values to compute and show

# Top and bottom envelopes
env1  = +A * np.exp(-a * x)  # top envelope
env2  = -A * np.exp(-a * x)  # bottom envelop

s = lambda x, t : A * np.exp(-a * x) * np.sin(omega * t - a * x + theta)
Q = lambda x, t : kD * A * a * np.exp(-a * x) * (np.sin(omega * t - a * x + theta) + np.cos(omega * t - a * x + theta))

# Setting up a nice plot
fig, axs = plt.subplots(3, 1, sharex=False, sharey=False)
fig.set_size_inches(12, 15)
axs[0].set_title("Head at varous times as a function of x")
axs[1].set_title("Head at various x as a function of time")
axs[2].set_title("Discharge at various x as s function of time")
axs[0].set_xlabel("x [m]")
axs[1].set_xlabel("time relative to the vertical axis [d]")
axs[2].set_xlabel("time relative to the vertical axis [d]")
axs[0].set_ylabel("head change s [m]")
axs[1].set_ylabel("head change s [m]")
axs[2].set_ylabel("discharge Q [m2/d]")
axs[0].grid()
axs[1].grid()
axs[2].grid()
 
# plot the envelopes
axs[0].plot(x, env1, 'b', label='top envelope')
axs[0].plot(x, env2, 'b', label='botom envelope')

# choose values for time
times1 = np.linspace(0, 1, 11)[:-1] * Tc
times2 = np.linspace(0, 1, 101)[:-1] * Tc

lines=[]

# do for each time, plot a wave
def plotall(i, dt, update=False):
    k = 0
    for ti in times1:        
        if not update:
            line, = axs[0].plot(x, s(x, ti +  i * dt), label=f't = {ti + dt:.2f}')
            lines.append(line)
        else:
            lines[k].set_data(x, s(x, ti +  i * dt))
        k += 1

    times = times2 + i * dt
    for xi in np.arange(20, 1500, 200):
        if not update:
            line, =axs[1].plot(times2 , s(xi, times), label='x = {:4.0f} m'.format(xi))
            lines.append(line)
        else:
            lines[k].set_data(times2, s(xi, times))
        k += 1
        if not update:
            line, =axs[2].plot(times2, Q(xi, times), label='x = {:.04f} m'.format(xi))
            lines.append(line)
        else:
            lines[k].set_data(times2, Q(xi, times))
        k += 1
    return lines


plotall(0, 0, False)

axs[0].legend(loc="upper right")
axs[1].legend(loc="upper right")
axs[2].legend(loc="upper right")

def init():
    for line in lines:
        line.set_data([], [])
    return lines

def animate(i, dt, update):
    plotall(i, dt, update)
    return lines

anim = FuncAnimation(fig, animate, frames=100, fargs=(Tc / 100, True), init_func=init,
                        interval=100, blit=True, repeat=True)

print('Done!')

plt.close(anim._fig)
#out = HTML(anim.to_html5_video())
#out # to actually show the video.

Done!


In [229]:
fname = "SinesWavesHeadDischarge"
anim.save(fname + '.mp4', fps=15, extra_args=['-vcodec', 'libx264'])

In [230]:
!ffmpeg -i SinesWavesHeadDischarge.mp4 SinesWavesHeadDischarge.gif

ffmpeg version 3.2.1 Copyright (c) 2000-2016 the FFmpeg developers
  built with llvm-gcc 4.2.1 (LLVM build 2336.11.00)
  configuration: --prefix=/Volumes/Ramdisk/sw --enable-gpl --enable-pthreads --enable-version3 --enable-libspeex --enable-libvpx --disable-decoder=libvpx --enable-libmp3lame --enable-libtheora --enable-libvorbis --enable-libx264 --enable-avfilter --enable-libopencore_amrwb --enable-libopencore_amrnb --enable-filters --enable-libgsm --enable-libvidstab --enable-libx265 --disable-doc --arch=x86_64 --enable-runtime-cpudetect
  libavutil      55. 34.100 / 55. 34.100
  libavcodec     57. 64.101 / 57. 64.101
  libavformat    57. 56.100 / 57. 56.100
  libavdevice    57.  1.100 / 57.  1.100
  libavfilter     6. 65.100 /  6. 65.100
  libswscale      4.  2.100 /  4.  2.100
  libswresample   2.  3.100 /  2.  3.100
  libpostproc    54.  1.100 / 54.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'SinesWavesHeadDischarge.mp4':
  Metadata:
    major_brand     : isom
    minor_version

# Animations

In [ ]:
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from matplotlib.text import Text
from IPython.display import HTML

In [84]:
A, kD, S, T_cycle = 2.0, 600., 0.01, 1.0
omega = 2 * np.pi / T_cycle
a = np.sqrt((omega * S) / (2 * kD))
times = T_cycle * np.linspace(0, 1, 100)[:-1] # Times within a single cycle


y   = lambda x, t: np.exp(-a * x) * np.sin(omega * t - a * x)
env = lambda x: np.exp(-a * x)

fig, ax = newfig("Sine head wave. Cycle time = 24h, kD={} m2/d, S={} [-]".format(T_cycle, kD, S),
           "x [m]", "head [m]")

x = np.linspace(0, 500, 501)

ax.plot(x, +env(x), '--', lw=2, color='gray',      label="upper envelope")
ax.plot(x, -env(x), '--', lw=2, color='darkgray', label="lower envelope")

bbox = dict(boxstyle="round", fc="0.8")
ax.annotate('upper envelope', xy=(100, env(100)), ha='right', xytext=(0.4, 0.85), textcoords='axes fraction',
            fontsize=15, arrowprops=dict(arrowstyle='->'), bbox=bbox)
ax.annotate('lower envelope', xy=(100, -env(100)), ha='right', xytext=(0.4, 0.15), textcoords='axes fraction',
            fontsize=15, arrowprops=dict(arrowstyle='->'), bbox=bbox)

t = 0.
line, = ax.plot(x,  np.exp(-a * x) * np.sin(omega * t - a * x), label="head (t, x)")
txt   = ax.text(0.6, 0.85, '', transform=ax.transAxes, fontsize=15, bbox=bbox)
ax.legend(loc="upper right")

def init():
        line.set_data([], [])
        txt.set_text("")
        return line, txt
        
def animate(t, a, omega, x):
        line.set_data(x, np.exp(-a * x) * np.sin(omega * t - a * x))
        txt.set_text('t = {:6.3} d'.format(t))
        return line, txt

if True:
    anim = FuncAnimation(fig, animate, frames=times, fargs=(a, omega, x), init_func=init,
                        interval=20, blit=True, repeat=True)

    print('Done!')

    plt.close(anim._fig)
    out = HTML(anim.to_html5_video())
out # to actually show the video.

Done!


In [86]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=30, metadata=dict(artist='Me'), bitrate=-1,
                extra_args=['-vcodec', 'libx264'])

fname = "SingleSineWave"
anim.save(fname + '.mp4', writer=writer)

In [191]:
# Convert the mp4 into a self-repeating gif file
#!ffmpeg -i SingleSineWave.mp4 -y SingleSineWave.gif

# Superposition of a number of waves having different amplitudes, cycle times and values at t=0.

In [110]:
import pandas as pd

# Aquifer
A, kD, S, = 2.0, 600., 0.01

columns = ['A', 'T_cycle', 'beta', 'label']
data = [[0.4, 1,  0.0, ' 1 d wave'],
        [0.7, 3,  0.2, ' 3 d wave'],
        [0.5, 5,  0.1, ' 5 d wave']]
cdata = pd.DataFrame(data, columns=columns)


cdata['omega'] = 2 * np.pi / cdata['T_cycle']
cdata['a']     = np.sqrt((cdata['omega'] * S) / (2 * kD))
         
times = np.linspace(0, 15, 10 * 24)

def y(t=None, x=None, cdata=None, index=None):
    """Return wave or combined waves at t=t."""
    index = [index] if index is not None else cdata.index 
    y = np.zeros_like(x)
    for i in index:
        rec = cdata.loc[i]
        y += rec['A'] * np.exp(- rec['a'] * x) * np.sin(rec['omega'] * t - rec['a'] * x + rec['beta'])
    return y
         
def y_env(x, cdata=None):
    "Return top envelope of combined waves."
    if np.isscalar(x): x = np.array(x)
    env = np.zeros_like(x)
    for i in cdata.index:
         rec = cdata.loc[i]
         env = env + rec['A'] * np.exp(- rec['a'] * x)
    return env

fig, ax = newfig("Sine head waves. Aquifer: kD={} m2/d, S={} [-]".format(kD, S),
           "x [m]", "heads [m]")
x = np.linspace(0, 500, 501)
         
lines = []
for i in cdata.index:
         line, = ax.plot(x, y(0, x, cdata, i), label=cdata['label'].loc[i])
         lines.append(line)
         
tot_line, = ax.plot(x, y(0, x, cdata), color='black', lw=3, label="sum of waves")
         
ax.plot(x, + y_env(x, cdata), '--', lw=2, color='gray', label='top envelope')
ax.plot(x, - y_env(x, cdata), '--', lw=2, color='gray', label='bottom evel.')
ax.legend(loc="upper right")

txt   = ax.text(0.6, 0.85, '', transform=ax.transAxes, fontsize=15, bbox=bbox)

bbox = dict(boxstyle="round", fc="0.8")
ax.annotate('upper envelope', xy=(100, +y_env(100, cdata)), xytext=(0.4, 0.85), textcoords='axes fraction',
            fontsize=15, arrowprops=dict(arrowstyle='->'), bbox=bbox)
ax.annotate('lower envelope', xy=(100, -y_env(100, cdata)), xytext=(0.4, 0.15), textcoords='axes fraction',
            fontsize=15, arrowprops=dict(arrowstyle='->'), bbox=bbox)

def init():
        for line in lines:
            line.set_data([], [])
        tot_line.set_data([], [])
        txt.set_text("")
        return (*lines, tot_line, txt)
        
def animate(t, a, omega, x):
        for line, i in zip(lines, cdata.index):
            line.set_data(x, y(t=t, x=x, cdata=cdata, index=i))
        tot_line.set_data(x, y(t=t, x=x, cdata=cdata))
        txt.set_text('t = {:6.3f} d'.format(t))
        return (*lines, tot_line, txt)

if True:
    anim = FuncAnimation(fig, animate, frames=times, fargs=(a, omega, x), init_func=init,
                        interval=50, blit=True, repeat=True)

    print('Done!')

    plt.close(anim._fig)
    out = HTML(anim.to_html5_video())
out # to actually show the video.

Done!


In [111]:
fname = "MultipleSineWaves"
anim.save(fname + '.mp4', fps=15, extra_args=['-vcodec', 'libx264'])

!ffmpeg -i MultipleSineWaves.mp4 MultipleSineWaves.gif

NameError: name 'system' is not defined

In [192]:
#!ffmpeg -i MultipleSineWaves.mp4 MultipleSineWaves.gif

# Sine wave in context

In [195]:
def bbox2figure(x0=None, y0=None, h=None, w=None, ax=None):
    """Convert axes data position to axes fraction position."""
    xy = [[x0, y0], [x0 + h, y0 + w]]
    uvDisp = ax0.transData.transform(xy) # Data --> Display
    uvFig  = ax0.figure.transFigure.inverted().transform(uvDisp) # Display to figure
    return uvFig[0][0], uvFig[0][1], uvFig[1][0] - uvFig[0][0], uvFig[1][1] - uvFig[0][1]


Aquif = dict(A=10.0, kD=600, S=0.001, Tcycle=0.5)
Aquif['omega'] = 2 * np.pi / Aquif['Tcycle']
Aquif['a'] = np.sqrt(Aquif['omega'] * Aquif['S'] / 2 / Aquif['kD'])

hbase, haquif, haclude, hlake0= 5, 25, 25, 37.5
L0, L1 = 150, 1000
x = np.linspace(0, L1, 101)

def rect(x0=None, y0=None, w=None, h=None, **kw):
    
    return PathPatch(Path([[x0, y0],
                         [x0 + w, y0],
                         [x0 + w, y0 + h],
                         [x0, y0 + h],
                         [x0, y0]], codes=[1, 2, 2, 2, 79]), **kw)
                
def set_path(path, x0, y0, w, h):
    """Updat the height of a rectangle PathPatch."""
    path.vertices = np.array([[x0, y0], [x0 + w, y0], [x0 + w, y0 + h], [x0, y0 + h], [x0, y0]])
    return path
 
ddn = lambda t, x: Aquif['A'] * np.exp(-Aquif['a'] * x) * np.sin(Aquif['omega'] * t - Aquif['a'] * x)

params = {'base' : dict(x0=-L0, y0=0, w=L0 + L1, h=-hbase, fc='darkgreen'),
          'aquif': dict(x0=  0, y0=0, w=L1, h=haquif, fc='gold'),
          'aclud': dict(x0=  0, y0=haquif, w=L1, h=haclude, fc='brown'),
          'lake' : dict(x0=-L0, y0=0, w=L0, h=hlake0, fc='blue'),
         }

base    = rect(**params['base'])
aquif   = rect(**params['aquif'])
aclud   = rect(**params['aclud'])
lake    = rect(**params['lake'])

things = [base, aquif, aclud, lake]

fig, ax0 = plt.subplots(); fig.set_size_inches(12, 6)

for thing in things:
    ax0.add_patch(thing)
    
ax0.set_xlim((-L0, L1))
ax0.set_ylim((-hbase, haquif + haclude + 10))
ax0.set_fc("none")
bbox = dict(boxstyle="round", fc="0.8")
ax0.annotate('x', (0, 0.1 * haquif),
            ha='right', xytext=(L0, 0.1 * haquif), textcoords='data', va='center',
            fontsize=15, arrowprops=dict(arrowstyle='<-'))
ax0.annotate('', (  -L0/2, hlake0 - Aquif['A']),
            xytext=(-L0/2, hlake0 + Aquif['A']), textcoords='data', va='center',
            fontsize=15, arrowprops=dict(arrowstyle='<->'))

ax0.text(-L0/2, hlake0, '2A', ha='center', fontsize=15, bbox=bbox)

ax0.text(3 * L0, 0.6 * haquif, 'kD={:.0f} m2/d, S={:.2f} [-]'.format(Aquif['kD'], Aquif['S']), fontsize=15)

x = np.linspace(0, L1, 400)
tmax = Aquif['Tcycle']

frac = np.linspace(0,  1, 100)
times  =  frac * tmax
theta = 2 * np.pi * frac
dhlake = Aquif['A'] * np.sin(Aquif['omega'] * times)

ax1 = fig.add_subplot(position=bbox2figure(0, hlake0 - Aquif['A'], L1, 2 * Aquif['A'], ax=ax0))
ax1.set_xlim((0, L1))
ax1.set_ylim((-Aquif['A'], Aquif['A']))
ax1.set_fc("none")
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)

line, = ax1.plot(x, ddn(0, x), color='w')
txt = ax0.text(0, haquif + haclude + 5, "t = {:6.3f} d".format(0), fontsize=15, bbox=bbox)

def init():
    path = lake.get_path()
    path.vertices = []
    lake.set_path(path)
    line.set_data([], [])
    txt.set_text("")
    return lake, line, txt
def animate(t_hlake):
    t, h = t_hlake
    path = lake.get_path()
    lake.set_path(set_path(path, -L0, 0, L0, hlake0 + h))
    line.set_data(x, ddn(t, x))
    txt.set_text("t = {:6.2f} d".format(t))
    return lake, line, txt

if True:
    anim = FuncAnimation(fig, animate, frames=zip(times, dhlake), fargs=None, init_func=init,
                        interval=50, blit=True, repeat=True)

    plt.close(anim._fig)
    out = HTML(anim.to_html5_video())
out

In [188]:
fname = "SineWaveInContext"
anim.save(fname + '.mp4', fps=20, extra_args=['-vcodec', 'libx264'], bitrate=1000)
print(anim.save_count, " frames saved.") # Shows the number of frames saved.

100  frames saved.


In [193]:
#!ffmpeg -i SineWaveInContext.mp4 -y SineWaveInContext.gif

In [173]:
pwd

'/Users/Theo/Instituten-Groepen-Overleggen/IHE/git/TransientGroundwaterFlow/excercises_notebooks'